In [1]:
import openai
import os
from dotenv import dotenv_values, load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPEN_AI_API_KEY')

In [5]:
import pandas as pd

In [6]:
import numpy as np

In [8]:
df = pd.read_csv("dataset.csv")
df

,Word
0,Pen
1,Pencil
2,Car
3,Auto
4,Vehicle


In [12]:
client = openai.OpenAI(
    api_key=openai.api_key,
)

In [9]:
def get_embedding(text):
    embedding_response = client.embeddings.create(
        input = [text], 
        model="text-embedding-3-small"
    )
    return embedding_response.data[0].embedding

In [13]:
df["embedding"] = df["Word"].apply(lambda x: get_embedding(x))
df

,Word,embedding
0,Pen,"[0.04258342087268829, 0.004893449600785971, 0...."
1,Pencil,"[0.014844437129795551, 0.043353889137506485, 0..."
2,Car,"[0.008056452497839928, -0.007912316359579563, ..."
3,Auto,"[-0.004276655148714781, 0.024805273860692978, ..."
4,Vehicle,"[-0.022744473069906235, -0.016150211915373802,..."


In [14]:
df.to_csv("dataset_embeddings.csv", index=False)

In [19]:
df = pd.read_csv("dataset_embeddings.csv")
df

,Word,embedding
0,Pen,"[0.04258342087268829, 0.004893449600785971, 0...."
1,Pencil,"[0.014844437129795551, 0.043353889137506485, 0..."
2,Car,"[0.008056452497839928, -0.007912316359579563, ..."
3,Auto,"[-0.004276655148714781, 0.024805273860692978, ..."
4,Vehicle,"[-0.022744473069906235, -0.016150211915373802,..."


In [20]:
df["embedding"] = df["embedding"].apply(eval).apply(np.array)

In [21]:
df

,Word,embedding
0,Pen,"[0.04258342087268829, 0.004893449600785971, 0...."
1,Pencil,"[0.014844437129795551, 0.043353889137506485, 0..."
2,Car,"[0.008056452497839928, -0.007912316359579563, ..."
3,Auto,"[-0.004276655148714781, 0.024805273860692978, ..."
4,Vehicle,"[-0.022744473069906235, -0.016150211915373802,..."


In [24]:
search_word = input("Enter a a word to search: ")
search_vector = get_embedding(search_word)
print(search_vector)

Enter a a word to search:  Car


[0.008056452497839928, -0.007912316359579563, -0.031254786998033524, 0.06190268322825432, 0.017372198402881622, -0.03671678900718689, -0.04630563408136368, 0.046214599162340164, -0.0004471065476536751, 0.017220476642251015, -0.011811577714979649, -0.007756800390779972, 0.0064444029703736305, -0.00015753037587273866, -0.006554401479661465, -0.03526025265455246, -0.02108939364552498, 0.02339557185769081, 0.02849343977868557, -0.004119259305298328, 0.03164926543831825, 0.0407526008784771, -0.002340315841138363, -0.025686578825116158, 0.008693685755133629, -0.004502357915043831, 0.01517222635447979, 0.007529217284172773, -0.009770913980901241, -0.07337288558483124, 0.03929606452584267, -0.04135948792099953, -0.002425659680739045, -0.0441511794924736, -0.020497677847743034, 0.03013204038143158, 0.008185415528714657, 0.005226831883192062, -0.02653622254729271, 0.010150219313800335, -0.007210600655525923, 0.007218186743557453, 0.015020503662526608, 0.01940527744591236, 0.03953882306814194, 0.

In [25]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [26]:
df["cosine"] = df["embedding"].apply(lambda x:  cosine_similarity(x, search_vector))
df

,Word,embedding,cosine
0,Pen,"[0.04258342087268829, 0.004893449600785971, 0....",0.354648
1,Pencil,"[0.014844437129795551, 0.043353889137506485, 0...",0.283959
2,Car,"[0.008056452497839928, -0.007912316359579563, ...",1.000000
3,Auto,"[-0.004276655148714781, 0.024805273860692978, ...",0.500959
4,Vehicle,"[-0.022744473069906235, -0.016150211915373802,...",0.698711


In [27]:
df = df.sort_values("cosine", ascending=False)
df

,Word,embedding,cosine
2,Car,"[0.008056452497839928, -0.007912316359579563, ...",1.000000
4,Vehicle,"[-0.022744473069906235, -0.016150211915373802,...",0.698711
3,Auto,"[-0.004276655148714781, 0.024805273860692978, ...",0.500959
0,Pen,"[0.04258342087268829, 0.004893449600785971, 0....",0.354648
1,Pencil,"[0.014844437129795551, 0.043353889137506485, 0...",0.283959
